# Importing Dependencies

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

# Loading The Model

In [2]:
interpreter = tf.lite.Interpreter(model_path='singlepose-lightning.tflite')
interpreter.allocate_tensors()

# Making Detections

In [3]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print(keypoints_with_scores)
    
    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

[[[[0.55799073 0.578173   0.5389931 ]
   [0.51569605 0.60515475 0.51976216]
   [0.5084075  0.5301734  0.6153699 ]
   [0.5360844  0.5948321  0.5594719 ]
   [0.5319143  0.43714592 0.79523015]
   [0.7527387  0.7380409  0.6287837 ]
   [0.7677367  0.27801934 0.61691964]
   [0.9652499  0.86088383 0.15809457]
   [0.9488245  0.22176751 0.04195224]
   [0.88520163 0.86267084 0.03289451]
   [0.82316965 0.35846537 0.00732567]
   [1.022258   0.6768156  0.01750259]
   [1.0206532  0.31387487 0.00782899]
   [0.8950062  0.8620788  0.01184708]
   [0.8489636  0.4007599  0.00734232]
   [0.852298   0.9729655  0.01867823]
   [0.8519766  0.60155183 0.00390017]]]]


NameError: name 'draw_connections' is not defined

# Drawing Keypoints

In [ ]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

In [ ]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

# Drawing Edges

In [ ]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)